<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: PLBART Finetuning**

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [4]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [5]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(1000))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 1000
Test dataset size: 1000


In [6]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [7]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [8]:
max_tok_length = 512

from transformers import AutoTokenizer

checkpoint = "uclanlp/plbart-large"

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    truncation=True,
    max_length=max_tok_length,
    )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/986k [00:00<?, ?B/s]

In [9]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [10]:
train_dataset_filtered = train_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
validation_dataset_filtered = validation_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

Discarding source and target sentences with more than 512 tokens:   0%|          | 0/200 [00:00<?, ? examples/…

Discarding source and target sentences with more than 512 tokens:   0%|          | 0/1000 [00:00<?, ? examples…

Discarding source and target sentences with more than 512 tokens:   0%|          | 0/1000 [00:00<?, ? examples…

In [11]:
print("Training dataset size:", len(train_dataset_filtered))
print("Validation dataset size:", len(validation_dataset_filtered))
print("Test dataset size:", len(test_dataset_filtered))

Training dataset size: 192
Validation dataset size: 931
Test dataset size: 930


In [12]:
tokenized_train_dataset = train_dataset_filtered.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset_filtered.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

Map:   0%|          | 0/931 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

In [13]:
dic = {}
for sample in tokenized_train_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

37   1
38   1
39   1
40   1
41   2
44   1
45   1
48   3
49   4
50   1
51   1
52   1
53   1
56   3
57   2
58   1
60   1
61   1
63   1
64   1
65   1
66   4
68   1
69   1
72   2
73   2
74   2
75   4
77   2
79   2
80   1
81   2
82   1
85   1
86   1
88   3
89   1
90   1
92   1
93   1
94   4
95   1
96   1
97   1
98   4
102   1
104   1
105   1
107   2
108   1
109   1
110   1
111   1
113   1
115   1
117   1
119   1
120   1
121   2
123   1
124   1
126   2
127   2
128   1
129   1
131   2
133   1
135   1
137   1
138   2
139   1
140   1
141   1
142   4
143   1
148   1
151   1
153   2
154   1
157   1
158   1
159   2
160   1
161   1
166   1
167   1
168   1
170   1
172   1
173   1
174   1
177   2
178   2
180   1
182   1
189   2
201   1
208   1
210   1
219   1
220   1
221   1
226   1
232   2
237   1
239   1
242   1
244   1
246   1
250   1
259   1
260   2
265   1
267   2
271   2
280   1
282   1
283   1
295   2
298   1
303   1
305   1
308   1
314   1
316   1
328   1
330   1
332   1
333   1
335   1
338  

In [14]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

In [16]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False, gradient_checkpointing_kwargs={'use_reentrant':False})

In [17]:
from evaluate import load

metric = load("sacrebleu")

In [18]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    if isinstance(preds, np.ndarray):
        preds = torch.tensor(preds)

    preds = torch.clamp(preds, min=0, max=tokenizer.vocab_size - 1)
    decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)

    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j<0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### **Hyperparameter Search**

In [20]:
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00


In [23]:
import optuna
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
from peft import PrefixTuningConfig, get_peft_model

def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [2, 4, 8, 16])
    # lora_r = trial.suggest_categorical("lora_r", [8, 16, 32])
    # lora_alpha = trial.suggest_int("lora_alpha", 16, 64, step=16)
    # lora_dropout = trial.suggest_uniform("lora_dropout", 0.0, 0.3)
    prefix_length = trial.suggest_int("prefix_length", 10, 50, step=10)

    # config = LoraConfig(
    #     task_type="SEQ_2_SEQ_LM",
    #     r=lora_r,
    #     lora_alpha=lora_alpha,
    #     target_modules=["q_proj", "v_proj"],
    #     lora_dropout=lora_dropout,
    #     bias="none",
    # )
    config = PrefixTuningConfig(
        task_type="SEQ_2_SEQ_LM",
        num_virtual_tokens=prefix_length,
    )

    # lora_model = get_peft_model(model, config)
    prefix_model = get_peft_model(model, config)

    data_collator = DataCollatorForSeq2Seq(
                      tokenizer=tokenizer,
                      model=prefix_model,
                      pad_to_multiple_of=8
                      )

    args = Seq2SeqTrainingArguments(
        output_dir="./optuna_results_A2",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        predict_with_generate=True,
        logging_dir="./logs",
        logging_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        weight_decay=0.01
    )

    trainer = Seq2SeqTrainer(
              prefix_model,
              args,
              train_dataset=tokenized_train_dataset,
              eval_dataset=tokenized_validation_dataset,
              processing_class=tokenizer,
              data_collator=data_collator,
              compute_metrics=compute_metrics
              )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]

In [24]:
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 5)

[I 2025-01-27 02:30:08,030] A new study created in memory with name: no-name-a580a576-9304-4532-bbac-70b7f5cd5575
<ipython-input-23-e92044433888>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

[I 2025-01-27 03:13:18,644] Trial 2 finished with value: 2.1420631408691406 and parameters: {'learning_rate': 0.000892175138111727, 'batch_size': 16, 'prefix_length': 10}. Best is trial 0 with value: 1.8085308074951172.
<ipython-input-23-e92044433888>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.482200,3.183099,1.507600,20.993600
2,3.461300,3.156267,1.499700,20.993600
3,3.397400,3.147063,1.506400,20.993600


[I 2025-01-27 03:25:16,203] Trial 3 finished with value: 3.1470625400543213 and parameters: {'learning_rate': 3.8266844537007035e-05, 'batch_size': 16, 'prefix_length': 50}. Best is trial 0 with value: 1.8085308074951172.
<ipython-input-23-e92044433888>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.240200,2.450175,3.373600,21.000000
2,3.072800,2.315376,3.574100,21.000000
3,2.873500,2.273169,3.640900,21.000000


[I 2025-01-27 03:39:23,532] Trial 4 finished with value: 2.2731688022613525 and parameters: {'learning_rate': 0.00021250263504029782, 'batch_size': 8, 'prefix_length': 20}. Best is trial 0 with value: 1.8085308074951172.


In [25]:
print("Best hyperparameters: ", study.best_params)

Best hyperparameters:  {'learning_rate': 0.00022685046590385107, 'batch_size': 4, 'prefix_length': 30}


In [26]:
best_params = study.best_params

final_config = PrefixTuningConfig(
        task_type="SEQ_2_SEQ_LM",
        num_virtual_tokens=best_params['prefix_length'],
    )

In [27]:
final_model = get_peft_model(model, final_config)
final_model.print_trainable_parameters()

trainable params: 737,280 || all params: 406,762,496 || trainable%: 0.1813


In [28]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=final_model,
    pad_to_multiple_of=8
    )

### **Training**

In [29]:
final_args = Seq2SeqTrainingArguments(
    output_dir="./final_model_a2",
    run_name = 'best_model_run_a2',
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./final_logs",
    load_best_model_at_end=True,
    weight_decay=0.01
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from transformers import Seq2SeqTrainer

final_trainer = Seq2SeqTrainer(
    final_model,
    final_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [31]:
final_trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.079158,1.534200,21.000000
2,No log,1.783787,2.010900,20.998900
3,No log,1.636829,2.388300,21.000000
4,No log,1.564179,2.709300,21.000000
5,No log,1.542117,2.758000,21.000000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=240, training_loss=2.2797815958658854, metrics={'train_runtime': 1403.7339, 'train_samples_per_second': 0.684, 'train_steps_per_second': 0.171, 'total_flos': 569745344299008.0, 'train_loss': 2.2797815958658854, 'epoch': 5.0})

### **Evaluación**

In [32]:
test_batch_size = 32
batch_tokenized_test = tokenized_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/930 [00:00<?, ? examples/s]

In [33]:
number_of_batches = len(batch_tokenized_test["code"])
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(batch_tokenized_test["code"][i], max_length=max_tok_length,
                       truncation=True, return_tensors="pt", padding=True)
    output_batch = final_model.generate(input_ids=inputs["input_ids"].cuda(),
                                       attention_mask=inputs["attention_mask"].cuda(),
                                       max_length = max_tok_length, num_beams=2, do_sample=False,)
    output_sequences.extend(output_batch.cpu())

In [34]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [35]:
references = tokenizer.batch_decode(tokenized_test_dataset["labels"], skip_special_tokens=True)

In [36]:
sources = test_dataset_filtered['code']

In [37]:
decoded_preds[0]

'build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs, self.build_attrs, self.build_attrs, self.build_attrs, self.build_attrs, self.extra_attrs, extra_attrs, extra_attrs, extra_attrs, extra_attrs = None))): self.build_attrs(self.build_attrs(self.build_attrs, extra_attrs, extra_attrs, extra_attrs=self.extra_attrs, extra_attrs=self.extra_attrs)) ) ) ) ) ) ) return self.build_attrs(self.build_attrs(self.build_attrs(self.build_attrs, self.build_attrs, self.build_attrs(self.build_attrs, self.build_attrs(self.build_attrs, self.build_attrs, extra_attrs, extra_attrs=self.build_attrs, extra_attrs=self.build_attrs, extra_attrs) ) ) ) def build_attrs(self, base_attrs, base_attrs, extra_attrs, extra_attrs = None): self.build_attrs(self, base_attrs, extra_attrs, extra_attrs = self.build_attrs(self.build_attrs, extra_attrs, extra_attrs=extra_attrs) ) def build_attrs(self, 

In [38]:
references[0]

'Build an attribute dictionary.'

In [39]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [40]:
from evaluate import load

metric = load("sacrebleu")

In [41]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 4.4


In [42]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [43]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [44]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 40.16 %


In [45]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [46]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 12.63 %
Rouge 2: 8.03 %
Rouge L: 11.19 %
